# SEPTA DELAY ANALYSIS

# Introduction

Approximately 119,000 daily riders take advantage SEPTA’s Regional Rail system, which provides service to the Philadelphia metropolitan area. The SEPTA (Southeastern Pennsylvania Transportation Authority) Regional Rail system consists of commuter rail service on 13 branches to more than 150 active stations in Philadelphia, Pennsylvania, and its suburbs and satellite cities.


SEPTA reports On-Time Performance (OTP) to measure service reliability. OTP identifies the number of trains for all rail lines that arrive at their scheduled destination at the scheduled time. However, by industry standard, a train may arrive up to 5 minutes and 59 seconds after its scheduled time and still be considered on-time. SEPTA has set an On-Time Performance target such that 91% of its trains arrive on time. Thus, even with 100% “on time” performance, trains may still arrive late, forcing commuters to deal with uncertainty and lost time – especially if they rely on back-to-back connections.

Here is the [main dataset](https://www.kaggle.com/septa/on-time-performance)

In [1]:
# Mounting Google Drive
from google.colab import drive

drive.mount('/content/gdrive')
path = '/content/gdrive/My Drive/545/Project_545'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!unzip '/content/gdrive/My Drive/545/Project_545/on-time-performance.zip' 

Archive:  /content/gdrive/My Drive/545/Project_545/on-time-performance.zip
  inflating: database.sqlite         
  inflating: otp.csv                 
  inflating: trainView.csv           


In [3]:
# IMPORTS
import numpy as np
import pandas as pd
import seaborn as sns
import math
import sqlite3

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.figure_factory as ff

from wordcloud import WordCloud
from wordcloud import STOPWORDS

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Exploring SEPTA tweets

In [4]:
# installing required packages
!pip3 install -qq twint
!pip install -qq whatthelang

#twitter imports
import json
import csv
from datetime import date
from datetime import datetime
import time
import twint

     |████████████████████████████████| 1.2MB 24.7MB/s 
     |████████████████████████████████| 245kB 52.1MB/s 
     |████████████████████████████████| 92kB 11.3MB/s 
     |████████████████████████████████| 256kB 63.0MB/s 
     |████████████████████████████████| 153kB 53.8MB/s 
     |████████████████████████████████| 235kB 55.0MB/s 
     |████████████████████████████████| 788kB 8.0MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 245kB 22.0MB/s 


## TWINT API

[Twint](https://github.com/twintproject/twint) is an advanced Twitter scraping tool written in Python that allows for scraping Tweets from Twitter profiles without using Twitter's API. I used this tool to get all tweets from [@SEPTA](https://twitter.com/SEPTA), i.e. SEPTAs official twitter account.



In [0]:
# Instantiate and configure the twint-object
c = twint.Config()
c.Store_object = True
c.Pandas =True
c.Limit = 1000
c.Lang = 'en'
c.User_id = 16358930
c.Year = 2015

In [0]:
# Run search
twint.run.Search(c)

In [0]:
# Quick check
twint.storage.panda.Tweets_df.head(20)

In [0]:
# Quick Preprocessing tweets df
tweets = twint.storage.panda.Tweets_df.drop_duplicates(subset=['id'])
tweets.index = range(len(tweets))
septa_tweets = tweets[tweets['username'] == 'SEPTA']

In [0]:
# Saving the csv file
septa_tweets.to_csv('septa_tweets.csv',index = False, header=True)

## Working on tweets data EDA

In [0]:
# Twitter Dataset 
septa_tweets = pd.read_csv('/content/gdrive/My Drive/545/Project_545/septa_tweets.csv')
septa_tweets = septa_tweets[(septa_tweets['date'] > '2016-03-22 00:00:00') & (septa_tweets['date'] <= '2016-11-05 23:58:02')]

In [0]:
septa_tweets = septa_tweets.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'id',
                                            'conversation_id', 'created_at', 'reply_to'])


In [75]:
septa_tweets.head()

,date,tweet,day,hour,nlikes,nreplies,nretweets
0,2016-11-05 23:58:02,Newark: Train #231 going to Marcus Hook is ope...,6,23,0,0,0
1,2016-11-05 23:49:42,Newark: Outbound train #229 is experiencing de...,6,23,0,0,0
2,2016-11-05 23:40:04,West Trenton: Train #3563 to Thorndale estimat...,6,23,0,0,0
3,2016-11-05 23:28:03,Elwyn: Train #3536 to Doylestown estimated to ...,6,23,0,0,0
4,2016-11-05 23:24:18,Thorndale: UPDATE: Trains #3559 due to a crew ...,6,23,0,0,0


In [0]:
# print some random tweets
from random import randint
for i in range(20):
  row_num = randint(0, 10000)
  print(septa_tweets.iloc[row_num]['tweet'])

Elwyn: Outbound train #9365 is experiencing delays of up to 30 minutes due to mechanical problems.
Trenton: Train #9706 is experiencing delays of up to 30 minutes due to equipment problems in Trenton.
Thorndale: Train #1581 going to Bryn Mawr is operating 83 minutes late. Last at Overbrook.
Thorndale: Train #583 going to Thorndale is operating 12 minutes late. Last at Fern Rock TC.
Thorndale: Train #1541 going to Thorndale is operating 18 minutes late. Last at Suburban Station.
Fox Chase: Train #818 going to Fox Chase is operating 22 minutes late. Last at 30th St.
Thorndale: Train #9532 will operate express from Haverford to Center City due overcrowding.
Elwyn: Train #3522 to Doylestown estimated to be 10 minutes late, scheduled to depart Elwyn Station at 12:32PM.
Doylestown: Train #6572 going to Doylestown is operating 11 minutes late. Last at Fortuna.
West Trenton: Train #3563 to Thorndale estimated to be 10 minutes late, scheduled to depart West Trenton at 8:02PM.
Chestnut Hill West

#### Interesting format of SEPTA tweets, some common identifiable patterns...
 

*   ':' seperates line name and message
*   'Train ' precedes train number starting with '#'
*   ' minutes' string succeedes the time maybe late or delays
*   'Last at ' precesdes last visited station 



In [0]:
row_num = randint(0, 10000)
given_string = septa_tweets.iloc[row_num][1]
split_string = given_string.split(':', 1)
line_name, message = split_string[0], split_string[1]

print("for the row number -> ", row_num)
print('message - > ', message)
print("")
print('line_name - > ', line_name)


try:
  start = message.find("Train #") + len("Train #")
  train_id = int(message[start:].split(None, 1)[0])
  print('train_id - > ', train_id)
except:
  print('No Train Id , Its a bus or MFL or BSL or Special Message')

try:
  minutes_late_message = message.find(" minutes")
  minutes_late_after_split = int(message[:minutes_late_message].rsplit(None, 1)[1])
  print('minutes_late_after_split - > ', minutes_late_after_split)

except:
  print('No minutes inthe line')


try:
  last_at_string = message.split(' Last at ', 1)
  mini_message, last_station_with_dot = last_at_string[0], last_at_string[1]
  last_station = last_station_with_dot[:-1]
  print('last_station - > ', last_station)
except:
  depart = message.find("scheduled to depart ") + len("scheduled to depart ")
  end = message.find(" at ")
  last_station = message[depart:end]
  if (len(last_station) < 15):
    print('last_station - > ', last_station)
  else:
    print('Exception')

for the row number ->  6661
message - >   Train #6348 going to Neshaminy Falls is operating 10 minutes late. Last at Meadowbrook.

line_name - >  West Trenton
train_id - >  6348
minutes_late_after_split - >  10
last_station - >  Meadowbrook


In [0]:
class twitter_processing:

  def __init__(self):

    self.line_name = None
    self.message = None
    self.train_id = None
    self.minutes_late_after_split = None
    self.last_station = None



  def find_line_name(self, input_str):
    try:
      split_string = input_str.split(':', 1)
      self.line_name, self.message = split_string[0], split_string[1]
      return self.line_name
    except:
      return "NA"

  def find_message(self,input_str):
    try:
      split_string = input_str.split(': ', 1)
      self.line_name, self.message = split_string[0], split_string[1]
      return self.message
    except:
      return "NA"


  def find_train_id(self, input_str):
    self.message = self.find_message(input_str)
    if (self.message != 'NA'):
      try:
        start = self.message.find("Train #") + len("Train #")
        self.train_id = None
        self.train_id = int(self.message[start:].split(None, 1)[0])
        return self.train_id
      except:
        return 'NA'
    else:
      return 'NA'


  def find_delay_time(self, input_str):
    self.message = self.find_message(input_str)
    if (self.message != 'NA'):
      try:
        minutes_late_message = self.message.find(" minutes")
        self.minutes_late_after_split = None
        self.minutes_late_after_split = int(self.message[:minutes_late_message].rsplit(None, 1)[1])
        return self.minutes_late_after_split
      except:
        return "NA"
    else:
      return "NA"

  def find_last_station(self, input_str):
    self.message = self.find_message(input_str)
    if (self.message != 'NA'):
      try:
        last_at_string = self.message .split(' Last at ', 1)
        mini_message, last_station_with_dot = last_at_string[0], last_at_string[1]
        self.last_station = None
        self.last_station = last_station_with_dot[:-1]
        return self.last_station
      except:
        depart = self.message.find("scheduled to depart ") + len("scheduled to depart ")
        end = self.message.find(" at ")
        self.last_station = None
        self.last_station = self.message[depart:end]
        if (len(self.last_station) < 20):
          return self.last_station
        else:
          return 'NA'
    else:
      return "NA"


twt = twitter_processing()

septa_tweets['line_name'] = septa_tweets['tweet'].apply(lambda x : twt.find_line_name(x))
septa_tweets['train_id'] = septa_tweets['tweet'].apply(lambda x : twt.find_train_id(x))
septa_tweets['delay'] = septa_tweets['tweet'].apply(lambda x : twt.find_delay_time(x))
septa_tweets['last_station'] = septa_tweets['tweet'].apply(lambda x : twt.find_last_station(x))

In [0]:
exception_twts = septa_tweets[septa_tweets['line_name'] == 'NA']
all_exception_text = []
all_exception_text.append(str(exception_twts['tweet'].apply(lambda x : str(x))))
seperator = ', '
my_sep_str = seperator.join(all_exception_text)

In [10]:
septa_tweets.head()

,date,tweet,day,hour,nlikes,nreplies,nretweets,line_name,train_id,delay,last_station
0,2016-11-05 23:58:02,Newark: Train #231 going to Marcus Hook is ope...,6,23,0,0,0,Newark,231,17,Elm St
1,2016-11-05 23:49:42,Newark: Outbound train #229 is experiencing de...,6,23,0,0,0,Newark,NA,20,NA
2,2016-11-05 23:40:04,West Trenton: Train #3563 to Thorndale estimat...,6,23,0,0,0,West Trenton,3563,11,West Trenton
3,2016-11-05 23:28:03,Elwyn: Train #3536 to Doylestown estimated to ...,6,23,0,0,0,Elwyn,3536,25,Elwyn Station
4,2016-11-05 23:24:18,Thorndale: UPDATE: Trains #3559 due to a crew ...,6,23,0,0,0,Thorndale,3561,NA,NA


In [0]:
# Exception Word cloud
stopwords = set(STOPWORDS)

wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(str(my_sep_str)) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

In [0]:
septa_tweets.shape

(27627, 11)

In [11]:
# drop these from main table  # ImP dont need exception twts in main dataset
new_septa_tweets = septa_tweets[septa_tweets.line_name != 'NA']
new_septa_tweets.shape

(27607, 11)

In [0]:
new_septa_tweets

#### Lets get some visualisation from tis data # new_septa_tweets

In [0]:
# Top 10 Regional Rail Lines occuring in septa tweets

important_lines = pd.DataFrame(new_septa_tweets.line_name.value_counts()[:10]).reset_index()



important_lines = important_lines.rename(columns={"index" : 'line_name', 'line_name' : 'tweet_count'})
fig = px.bar(important_lines, x='line_name', y='tweet_count',
             hover_data=['tweet_count'], color='tweet_count',
             height=400)



fig.update_layout(
    title="Count of tweets for various regional lines in Twitter data",
    xaxis_title="Regional Rail Lines",
    yaxis_title="Count of tweets for (March to November)",
    font=dict(
        family="Times, monospace",
        size=14,
        color="#7f7f7f"
    )
)

fig.show()

In [0]:
important_lines = new_septa_tweets[['day', 'line_name']].groupby(['line_name', 'day']).size().reset_index(name='counts')

# important_lines
import functools
def conjunction(*conditions):
    return functools.reduce(np.logical_or, conditions)

c_1 = important_lines.line_name == 'Thorndale'
c_2 = important_lines.line_name == 'Doylestown'
c_3 = important_lines.line_name == 'Elwyn'
c_4 = important_lines.line_name == 'Norristown'

to_plot = important_lines[conjunction(c_1,c_2,c_3,c_4)][['line_name', 'day', 'counts']]
to_plot['day'] = to_plot['day'].map({1 : 'Monday',
                                     2 : "Tuesday",
                                     3 : 'Wednesday',
                                     4 : 'Thursday',
                                     5 : 'Friday',
                                     6 : 'Saturday',
                                     7 : 'Sunday'
                                     }) 
fig = px.scatter(to_plot, x='line_name', y='counts', color='counts',
                facet_col='day', facet_col_wrap=4)
fig.show()

In [0]:
# Regional Line wise identified delay in tweets
# 1. drop rows having NA in delay column
new_delay_septa_tweets =  new_septa_tweets[new_septa_tweets.delay != 'NA']
new_delay_septa_tweets['delay'] = new_delay_septa_tweets['delay'].astype(float) 
new_delay_septa_tweets_to_plot = new_delay_septa_tweets[['line_name', 'delay']].groupby('line_name').mean().sort_values(by='delay').reset_index()

fig = px.bar(new_delay_septa_tweets_to_plot, x='line_name', y='delay',
             hover_data=['delay'], color='delay',
             height=400)

fig.update_layout(
    title="Average delays for various regional lines in Twitter data",
    xaxis_title="Regional Rail Lines",
    yaxis_title="Average Delay in Minutes",
    font=dict(
        family="Times, monospace",
        size=14,
        color="#7f7f7f"
    )
)
fig.show()

In [0]:


# Rte 125 is the only excetion look to be the case here

# # Add histogram dat
x1 = list(new_delay_septa_tweets[new_delay_septa_tweets.line_name == 'Thorndale']['delay'])
x2 = list(new_delay_septa_tweets[new_delay_septa_tweets.line_name == 'Doylestown']['delay'])
x3 = list(new_delay_septa_tweets[new_delay_septa_tweets.line_name == 'Elwyn']['delay'])
x4 = list(new_delay_septa_tweets[new_delay_septa_tweets.line_name == 'Norristown']['delay'])

counts, bins = np.histogram(x1, bins=range(5, 35, 1))
bins = 0.5*(bins[:-1] + bins[1:])

fig = px.bar(x=bins, y=counts, labels={'x':'Thorndale', 'y':'count'})
fig.show()

counts, bins = np.histogram(x2, bins=range(5, 35, 1))
bins = 0.5*(bins[:-1] + bins[1:])

fig = px.bar(x=bins, y=counts, labels={'x':'Doylestown', 'y':'count'})
fig.show()

counts, bins = np.histogram(x3, bins=range(5, 35, 1))
bins = 0.5*(bins[:-1] + bins[1:])

fig = px.bar(x=bins, y=counts, labels={'x':'Elwyn', 'y':'count'})
fig.show()

counts, bins = np.histogram(x4, bins=range(5, 35, 1))
bins = 0.5*(bins[:-1] + bins[1:])

fig = px.bar(x=bins, y=counts, labels={'x':'Norristown', 'y':'count'})
fig.show()

hist_data = [x1,x2,x3,x4]
group_labels = ['Thorndale', 'Doylestown', 'Elwyn', 'Norristown']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=[1, 1, 1, 1])
fig.show()

Delay in tweets follow some pattern, generally they like to tweet delay times in multiples of 5 , we can see bin for 10 minutes being the highest
and also for 15, 20, 25 relatively higher than their neighbours

In [0]:
line_useful = ['Thorndale', 'Doylestown', 'Elwyn', 'Norristown']

import functools
def conjunction(*conditions):
    return functools.reduce(np.logical_or, conditions)

c_1 = new_delay_septa_tweets.line_name == 'Thorndale'
c_2 = new_delay_septa_tweets.line_name == 'Doylestown'
c_3 = new_delay_septa_tweets.line_name == 'Elwyn'
c_4 = new_delay_septa_tweets.line_name == 'Norristown'

new_to_plot = new_delay_septa_tweets[conjunction(c_1,c_2,c_3,c_4)][['line_name', 'delay']]

fig = px.histogram(new_to_plot, x="line_name", y="delay", facet_row='line_name')
fig.show()

# new_delay_septa_tweets_to_plot[new_delay_septa_tweets_to_plot.line_name == 'Thorndale' and new_delay_septa_tweets_to_plot.line_name == 'Doylestown' ]

In [0]:
list(new_delay_septa_tweets[new_delay_septa_tweets.line_name == 'Thorndale']['delay'])

## visualization over

##### Implementation

In [0]:
# Get some stats for this preprocessing
new_septa_tweets[['line_name' , 'train_id', 'delay', 'last_station']].describe()

In [0]:
new_septa_tweets

 ## Planned to use this to pull Weather data
 But they sold this API to Apple and subsequently it closed its services recently on March 31,2020.


 link - https://github.com/ZeevG/python-forecast.io

#### Add extra columns dealing with date time data

In [0]:
septa_tweets['month'] = pd.DatetimeIndex(septa_tweets['date']).month
septa_tweets['day'] = pd.DatetimeIndex(septa_tweets['date']).day
# septa_tweets['hour'] = pd.DatetimeIndex(septa_tweets['date']).hour
septa_tweets['time'] = pd.DatetimeIndex(septa_tweets['date']).time


septa_tweets['second'] = septa_tweets['time'].apply(lambda x : str(x).split(':', 2)[2])  
septa_tweets['minute'] = septa_tweets['time'].apply(lambda x : str(x).split(':', 2)[1])    
septa_tweets['hour'] = septa_tweets['time'].apply(lambda x : str(x).split(':', 2)[0])    
septa_tweets = septa_tweets.drop(columns = ['date', 'tweet', 'time'])

In [0]:
septa_tweets = septa_tweets[septa_tweets['line_name'] != 'NA']
septa_tweets = septa_tweets[septa_tweets['train_id'] != 'NA']
septa_tweets = septa_tweets[septa_tweets['delay'] != 'NA']
septa_tweets = septa_tweets[septa_tweets['last_station'] != 'NA']

In [0]:
septa_tweets['day_of_week'] = septa_tweets.apply(lambda x : datetime.datetime(2016,x.month,x.day).weekday(), axis=1)
septa_tweets = septa_tweets.drop(columns = ['day', 'second'])
septa_tweets["min_scaled"]= septa_tweets["minute"].apply(lambda x : math.ceil(int(x) / 3))
septa_tweets = septa_tweets.drop(columns=['minute']) 

In [81]:
septa_tweets = septa_tweets[['month','day_of_week', 'hour', 'min_scaled', 'line_name',
                             'train_id', 'delay', 'last_station', 
                             'nlikes','nreplies', 'nretweets']]
septa_tweets

,month,day_of_week,hour,min_scaled,line_name,train_id,delay,last_station,nlikes,nreplies,nretweets
0,11,5,23,20,Newark,231,17,Elm St,0,0,0
2,11,5,23,14,West Trenton,3563,11,West Trenton,0,0,0
3,11,5,23,10,Elwyn,3536,25,Elwyn Station,0,0,0
6,11,5,23,6,Norristown,231,13,Elm St,0,0,0
7,11,5,23,3,Newark,229,20,Norristown TC,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
27622,3,1,11,4,Thorndale,9538,11,Thorndale,0,1,1
27623,3,1,03,15,Doylestown,596,22,Fern Rock TC,0,0,1
27624,3,1,03,10,Doylestown,596,12,Wayne Jct,0,0,1
27625,3,1,02,10,Elwyn,393,11,West Trenton,0,0,1


## Working on kaggle data EDA

In [13]:
# sqlite dataset


# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("/content/database.sqlite")
df = pd.read_sql_query("SELECT * from otp", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

con.close()

  train_id direction  ...   status            timeStamp
0      778         N  ...    1 min  2016-03-23 00:01:47
1      598         N  ...    1 min  2016-03-23 00:01:58
2      279         S  ...    2 min  2016-03-23 00:02:02
3      476         N  ...  On Time  2016-03-23 00:03:19
4      474         N  ...  On Time  2016-03-23 00:03:35

[5 rows x 7 columns]


In [14]:
otp_df.describe()

NameError: ignored

In [0]:
list_origin = list(otp_df.origin.unique())
list_nest_station = list(otp_df.next_station.unique())
origin_not_next_station = set(list_origin) - set(list_nest_station)
origin_not_next_station

#### Cleaning data



In [158]:
import re
!pip install plotnine  
from pandas.api.types import CategoricalDtype
from plotnine import *
%matplotlib inline

def status_modify(given_str):

  if (str(given_str) == 'On Time'):
    return str(given_str).replace("On Time", '0')
  else:
    return str(given_str).replace(" min", '')

In [0]:
# Clean train_id 
otp_df = pd.read_csv('/content/otp.csv')
otp_df['train_id'] = otp_df['train_id'].apply(lambda x : re.sub("[^0-9/-/.]", "", str(x)))
otp_df['status_modified_minutes'] = otp_df['status'].apply(lambda x : status_modify(str(x)) )
otp_df['status_modified_minutes'] = otp_df['status_modified_minutes'].astype(int)
otp_df = otp_df.drop(columns= 'status')

# conditional processing
otp_nona =  otp_df[otp_df.origin != 'None']
otp_nona =  otp_nona[otp_nona.next_station != 'None']
## otp N is even train ids odd is South

In [181]:
dict(otp_nona[otp_nona.train_id == '476'].origin.value_counts())
# otp_df[otp_df.train_id == '418' otp_df.origin == '60th St North']

import functools
def conjunction(*conditions):
    return functools.reduce(np.logical_and, conditions)

c_1 = otp_df.train_id == '464'
c_2 = otp_df.date == '2016-04-08'

otp_df[conjunction(c_1, c_2)]

,train_id,direction,origin,next_station,date,timeStamp,status_modified_minutes
159357,464,N,None,None,2016-04-08,2016-04-08 20:37:16,0
159372,464,N,Airport Terminal E-F,Airport Terminal C-D,2016-04-08,2016-04-08 20:39:00,0
159375,464,N,Airport Terminal E-F,Airport Terminal B,2016-04-08,2016-04-08 20:39:16,0
159411,464,N,Airport Terminal E-F,Eastwick Station,2016-04-08,2016-04-08 20:45:17,1
159593,464,N,Airport Terminal E-F,Jefferson Station,2016-04-08,2016-04-08 21:08:20,1
159629,464,N,Airport Terminal E-F,Temple U,2016-04-08,2016-04-08 21:13:43,0
159698,464,N,Airport Terminal E-F,Wayne Jct,2016-04-08,2016-04-08 21:21:43,0
159757,464,N,Airport Terminal E-F,Fern Rock TC,2016-04-08,2016-04-08 21:29:43,5
159775,464,N,Airport Terminal E-F,Melrose Park,2016-04-08,2016-04-08 21:31:44,5
159785,464,N,Airport Terminal E-F,Elkins Park,2016-04-08,2016-04-08 21:33:19,4


In [182]:
otp_df.head()

,train_id,direction,origin,next_station,date,timeStamp,status_modified_minutes
0,778,N,Trenton,Stenton,2016-03-23,2016-03-23 00:01:47,1
1,598,N,Thorndale,Narberth,2016-03-23,2016-03-23 00:01:58,1
2,279,S,Elm,Ridley Park,2016-03-23,2016-03-23 00:02:02,2
3,476,N,Airport Terminal E-F,Suburban Station,2016-03-23,2016-03-23 00:03:19,0
4,474,N,Airport Terminal E-F,Jenkintown-Wyncote,2016-03-23,2016-03-23 00:03:35,0


In [0]:
import datetime
otp_date_df= otp_nona['date'].str.split("-", n = 2, expand = True) 

otp_nona["month"]= otp_date_df[1].astype(int) 
otp_nona["day"]= otp_date_df[2].astype(int) 
data_df = None

otp_nona['day_of_week'] = otp_nona.apply(lambda x : datetime.datetime(2016,x.month,x.day).weekday(), axis=1)
otp_df = otp_nona
otp_time_df = otp_df['timeStamp'].str.split(" ", n = 1, expand = True) 

In [0]:
otp_df['time'] = otp_time_df[1].astype(str) 
otp_time_df = None
otp_times_df= otp_df['time'].str.split(":", n = 2, expand = True)
otp_df["sec"]= otp_times_df[2].astype(int) 
otp_df["min"]= otp_times_df[1].astype(int) 
otp_df["hour"]= otp_times_df[0].astype(int) 
otp_df = otp_df.drop(columns=['date', 'timeStamp', 'time','sec']) 
otp_df["min_scaled"]= otp_df["min"].apply(lambda x : math.ceil(int(x) / 3))
otp_df = otp_df.drop(columns=['min']) 

In [185]:
otp_df.shape

(1793682, 10)

In [0]:
otp_df = otp_df[pd.to_numeric(otp_df['train_id'], errors='coerce').notnull()]
otp_df = otp_df[pd.to_numeric(otp_df['status_modified_minutes'], errors='coerce').notnull()]
otp_df = otp_df[pd.to_numeric(otp_df['day_of_week'], errors='coerce').notnull()]
otp_df = otp_df[pd.to_numeric(otp_df['hour'], errors='coerce').notnull()]
otp_df = otp_df[pd.to_numeric(otp_df['min_scaled'], errors='coerce').notnull()]

# check if origin , next_station, direction is not '', or Null, none, na, etc

In [0]:
counts = otp_df.train_id.value_counts()

otp_df_new = otp_df[~otp_df['train_id'].isin(counts[counts < 1000].index)]
otp_df_new = otp_df_new[['month','day', 'day_of_week', 'hour', 'min_scaled', 'origin',
                         'direction','train_id', 'status_modified_minutes', 'next_station']]



In [0]:
otp_df_new.to_csv('otp_df_new.csv',index = False, header=True)

In [0]:
otp_df_new.head()

In [0]:
septa_tweets.head()

In [0]:
weather_df = pd.read_csv('/content/gdrive/My Drive/545/Project_545/temp_phl.csv', header='infer')

weather_df['day'] = pd.DatetimeIndex(weather_df['Day']).day
weather_df['month'] = pd.DatetimeIndex(weather_df['Day']).month
weather_df['year'] = 16
weather_df = weather_df.drop(columns='Day')
weather_df = weather_df.dropna()
weather_df['day'] = weather_df['day'].astype(int)
weather_df['month'] = weather_df['month'].astype(int)

weather_df.to_csv('weather_df.csv',index = False, header=True)

weather_df.head()


## ML here

#### ml imports

In [0]:
! sudo apt install openjdk-8-jdk
! sudo update-alternatives --config java
import glob
import seaborn as sns
import re
import os

In [0]:
!apt install libkrb5-dev
!wget https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install findspark
!pip install sparkmagic
!pip install pyspark
!pip install pyspark --user
!pip install seaborn --user
!pip install plotly --user
!pip install imageio --user
!pip install folium --user

In [0]:
!apt update
!apt install gcc python-dev libkrb5-dev

from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

import os

spark = SparkSession.builder.appName('545-project').getOrCreate()


In [0]:
%load_ext sparkmagic.magics

In [0]:
#graph section
import networkx as nx
# SQLite RDBMS
import sqlite3
# Parallel processing
# import swifter
import pandas as pd
# NoSQL DB
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError, OperationFailure

import os
os.environ['SPARK_HOME'] = '/content/spark-2.4.5-bin-hadoop2.7'
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
import pyspark
from pyspark.sql import SQLContext

In [0]:
try:
    if(spark == None):
        spark = SparkSession.builder.appName('Initial').getOrCreate()
        sqlContext=SQLContext(spark)
except NameError:
    spark = SparkSession.builder.appName('Initial').getOrCreate()
    sqlContext=SQLContext(spark)


#### imports over

##### not used

In [0]:
data_sdf = spark.read.csv(
    "/content/otp_df_new.csv" , header=True, inferSchema = True, sep = ',')
weather_sdf = spark.read.csv(
    "/content/weather_df.csv" , header=True, inferSchema = True, sep = ',')
cond = [data_sdf.month == weather_sdf.month, data_sdf.day == weather_sdf.day]

In [0]:
inner_join = data_sdf.join(weather_sdf, cond ,how='full')
columns_to_drop = ['year', 'day', 'month']

inner_join = inner_join.drop(*columns_to_drop)
inner_join = inner_join.na.drop()

In [0]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
inner_result_pdf = inner_join.toPandas()

In [0]:
X = inner_result_pdf.drop(columns = ['status_modified_minutes', 'next_station']).astype(object)
Y = inner_result_pdf[['status_modified_minutes']]

In [243]:
X.head(1)

,day_of_week,hour,min_scaled,origin,train_id,Max,Min,rain
0,2,0,0,Marcus Hook,9264,62,43,1.13


In [0]:
trial1_X = pd.get_dummies(X, columns=['day_of_week','hour','min_scaled','origin','train_id'])

In [0]:
columns_to_use = list(trial1_X.columns) + ['Max', 'Min', 'rain']

In [0]:
trial1_X_sdf = spark.createDataFrame(trial1_X)

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
    inputCols=columns_to_use,
    outputCol="features")

In [0]:
from pyspark.ml import Pipeline

# Your code goes here
pipeline = Pipeline(stages=[assembler])

pipe_model = pipeline.fit(trial1_X_sdf)
modified_data_sdf = pipe_model.transform(trial1_X_sdf)

In [0]:
train_sdf, test_sdf = modified_data_sdf.randomSplit([0.7, 0.3])

## ML

In [0]:
trial1_X = otp_df_dummies.head(20000)
trial1_Y = Y.head(20000)
trial1_Y = np.asarray(trial1_Y).flatten()

In [0]:
trial1_X_test = otp_df_dummies.tail(2000)
trial1_Y_test = Y.tail(2000)
trial1_Y_test = np.asarray(trial1_Y_test).flatten()

In [0]:
trial1_X_test

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
# Set the parameters by cross-validation
parameters = {'n_estimators':[100], 'max_depth': [10]}

model = RandomForestRegressor()
grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 5, verbose = True, scoring = 'r2')
grid.fit(trial1_X, trial1_Y)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_params_)

In [0]:
# x_test = scaler.transform(x_test)
# x_test = pca.fit_transform(x_test)
y_pred_train = grid.predict(trial1_X)
y_pred = grid.predict(trial1_X_test)

In [115]:
from sklearn.metrics import mean_squared_error
from math import sqrt
print(np.sqrt(mean_squared_error(trial1_Y, y_pred_train)))
print(np.sqrt(mean_squared_error(trial1_Y_test, y_pred)))

4.505304162683821
23.63837569217532


In [0]:
septa_tweets.head()

,date,tweet,day,hour,nlikes,nreplies,nretweets,line_name,train_id,delay,last_station
0,2016-11-05 23:58:02,Newark: Train #231 going to Marcus Hook is ope...,6,23,0,0,0,Newark,231,17,Elm St
1,2016-11-05 23:49:42,Newark: Outbound train #229 is experiencing de...,6,23,0,0,0,Newark,NA,20,NA
2,2016-11-05 23:40:04,West Trenton: Train #3563 to Thorndale estimat...,6,23,0,0,0,West Trenton,3563,11,West Trenton
3,2016-11-05 23:28:03,Elwyn: Train #3536 to Doylestown estimated to ...,6,23,0,0,0,Elwyn,3536,25,Elwyn Station
4,2016-11-05 23:24:18,Thorndale: UPDATE: Trains #3559 due to a crew ...,6,23,0,0,0,Thorndale,3561,NA,NA


In [0]:
trainView_df.head()

,train_id,status,next_station,service,dest,lon,lat,source,track_change,track,date,timeStamp0,timeStamp1,seconds
0,102TT,0,Radnor,LOCAL,Colmar-Link Belt,-75.37250,40.04388,Devon,-1,-1,2016-04-22,2016-04-22 13:21:07,2016-04-22 13:22:43,96
1,102TT,0,St. Davids,LOCAL,Colmar-Link Belt,-75.38670,40.04583,Devon,-1,-1,2016-04-22,2016-04-22 13:19:11,2016-04-22 13:21:01,110
2,102TT,0,Strafford,LOCAL,Colmar-Link Belt,-75.42277,40.04722,Devon,-1,-1,2016-04-22,2016-04-22 13:15:04,2016-04-22 13:17:01,117
3,102TT,0,Villanova,LOCAL,Colmar-Link Belt,-75.36040,40.04450,Devon,-1,-1,2016-04-22,2016-04-22 13:22:33,2016-04-22 13:22:33,0
4,102TT,0,Wayne-A,LOCAL,Colmar-Link Belt,-75.40447,40.04961,Devon,-1,-1,2016-04-22,2016-04-22 13:16:32,2016-04-22 13:19:06,154


## otp _plots

In [0]:
# # Add histogram dat
x1 = list(otp_df[otp_df.origin == 'Thorndale']['status_modified_minutes'])
x2 = list(otp_df[otp_df.origin == 'Doylestown']['status_modified_minutes'])
x3 = list(otp_df[otp_df.origin == 'Elwyn']['status_modified_minutes'])
x4 = list(otp_df[otp_df.origin == 'Elm St']['status_modified_minutes']) # Norristown line is name as Elm st in otp database, different from twitter


hist_data = [x1,x4]
group_labels = ['Thorndale', 'Norristown'] #, 'Elwyn', 'Norristown']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=[1, 1]) #, 1, 1])
fig.show()

In [0]:
len(x1),len(x2),len(x3),len(x4)

(87337, 136848, 92849, 16259)

In [0]:
(ggplot(agg_otp_df, aes(x='minutes', y='percentage')) +
  geom_line() +
  xlim(0,10))

In [0]:
(ggplot(agg_otp_df, aes(x='minutes', y='percentage')) +
  geom_line())

In [0]:
otp_df.describe()

#### Fun Map

In [0]:
!pip install folium

In [0]:
import folium

In [0]:
locations = trainView_df[['lat', 'lon']]
locationlist = locations.values.tolist()
len(locationlist)


3601656

In [0]:
small_location_list = locationlist[:1000]

In [0]:
map = folium.Map(location=[39.9526, -75.1652], zoom_start=12)
for point in range(0, len(small_location_list)):
    folium.Marker(small_location_list[point], popup=trainView_df['next_station'][point]).add_to(map)
map

# trainView_df Analysis

In [0]:
# given csv Datasets
trainView_df = pd.read_csv('/content/trainView.csv', low_memory=False)
trainView_df.head(2)

,train_id,status,next_station,service,dest,lon,lat,source,track_change,track,date,timeStamp0,timeStamp1,seconds
0,102TT,0,Radnor,LOCAL,Colmar-Link Belt,-75.3725,40.04388,Devon,-1,-1,2016-04-22,2016-04-22 13:21:07,2016-04-22 13:22:43,96
1,102TT,0,St. Davids,LOCAL,Colmar-Link Belt,-75.3867,40.04583,Devon,-1,-1,2016-04-22,2016-04-22 13:19:11,2016-04-22 13:21:01,110


In [0]:
trainView_df.value_counts()

##### WOrking with suspended trains

In [0]:
# status ('On Time', '5 min', '10 min'. This is a status on train lateness. 999 is a suspended train)
suspended_df = trainView_df[trainView_df.status == 999]
# Trains for the source-dest pair were suspended most
to_view_suspended = suspended_df[['source', 'dest']]
to_view_suspended['route'] = to_view_suspended['source'].astype(str) + str(' - ') + to_view_suspended["dest"].astype(str)
to_view_suspended = to_view_suspended.drop(columns = ['source', 'dest']).reset_index().drop(columns = ['index'])
agg_df_suspended = to_view_suspended.groupby('route').size().sort_values(ascending=False)[:20]
agg_df_suspended = pd.DataFrame(agg_df_suspended).reset_index().rename(columns = {0 : 'count'})

In [0]:

fig = px.bar(agg_df_suspended, x='route', y='count',
             hover_data=['count'], color='count',
             height=400)



fig.update_layout(
    title="Count of instances for suspended regional lines train services(TOP 20)",
    xaxis_title="Regional Rail Routes (Source - Destination)",
    yaxis_title="Count of suspended journeys for (March to November)",
    font=dict(
        family="Times, monospace",
        size=12,
        color="#7f7f7f"
    )
)

fig.show()

#### train_df processing


In [0]:
# trainView_df['route'] = trainView_df['source'].astype(str) + str(' - ') + trainView_df["dest"].astype(str)
trainView_df['dest'].nunique(), trainView_df['source'].nunique()

(59, 157)

In [0]:
# Feature Engineering
import datetime
import calendar

date_df= trainView_df['date'].str.split("-", n = 2, expand = True) 

trainView_df["month"]= date_df[1].astype(int) 
trainView_df["day"]= date_df[2].astype(int) 
data_df = None

trainView_df['day_of_week'] = trainView_df.apply(lambda x : datetime.datetime(2020,x.month,x.day).weekday(), axis=1)
# calendar.day_name[datetime.datetime(2020,5,4).weekday()]


In [0]:
trainView_df['day_name'] = trainView_df['day_of_week'].map({0 : 'Monday',1 : "Tuesday",2 : 'Wednesday',
                                                            3 : 'Thursday',4 : 'Friday',5 : 'Saturday',
                                                            6 : 'Sunday'})

In [0]:
trainView_df.track_change.value_counts()

-1            3278829
-1             258653
1               12369
4               11565
2               10566
3                7788
6                4343
A                3592
B                3502
5                3146
2016-05-25       2902
4                1311
2                 815
1                 675
3                 480
1A                150
6                 132
2B                130
4A                123
3A                109
1B                 95
5                  78
4B                 76
5A                 57
7                  54
2A                 48
2016-05-26         17
6A                 16
0                  15
3B                 13
5B                  6
55                  1
Name: track_change, dtype: int64

In [0]:

list_origin = list(trainView_df.source.unique())
list_nest_station = list(trainView_df.next_station.unique())
origin_not_dest = set(list_origin) - set(list_nest_station)
dest_not_origin = set(list_nest_station) - set(list_origin) 


## My Project / Analysis should look like ---
1. show all preprocessing, data gathering
2. twitter data analysis and feature preprocessing
3. Basic data (otp and trainview csv processing)
-----------------------------------------------------
4. Basic EDA related to our problem... should highlight the path to our model/ problem statement.

5. Think about feature engineering to make model more effective in predicting.
------------------------------------------------------
6. Give some perticular rnadom/amazing/peculiar insight from data related to our model

6. making df ready for the ML task.
-----------------------------------------------------------
7. Perform the ML task , compare algo, if multiple models developed. Also hyperparameter tuning,etc
---------------------------------------------------------
8. do model output analysis, what do we conclude from our study.


## key concepts learned in the class may be usefull in project!

